Operartis-1A KNN Algorithm

In [1]:
import pandas as pd
import numpy as np
import os 

In [2]:
filename = os.path.join(os.getcwd(), "train.csv")
df = pd.read_csv(filename, header=0)

In [3]:
df.head()


,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,A_debitOrCredit,A_amount,...,B_amount,B_valueDate,B_currencyCode,B_account,B_transactionReferences,B_transactionAttributes,B_orderingPartyInfo,B_receivingPartyInfo,targetAllocation,generatorAllocation
0,1287,9/18/13,MANUAL,Jane Doe,0,A,1291.0,9/18/13,NONE,5481.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/11/13_3446886485_600 cc AAB
1,1293,9/18/13,MANUAL,Jane Doe,0,A,1313.0,9/18/13,NONE,"34,233.97",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_211 AAB
2,1293,9/18/13,MANUAL,Jane Doe,0,A,1314.0,9/18/13,NONE,"32,643.27",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_209 AAB
3,1293,9/18/13,MANUAL,Jane Doe,0,A,1315.0,9/18/13,NONE,"36,897.54",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_205 AAB
4,1293,9/18/13,MANUAL,Jane Doe,0,A,1316.0,9/18/13,NONE,"31,444.64",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_202 AAB


In [9]:
df.shape

(223937, 31)

Splitting the DataFrames into two seprate dataframes one for Transaction A and one for Transaction B

In [4]:
#List of all columns in the dataframe 
name_of_columns = df.columns.tolist()
print(name_of_columns)

['matchId', 'matchDate', 'matchRule', 'matchedBy', 'wasPreviouslyMismatched', 'generatorArtifactName', 'generatorId', 'generatorImportDate', 'A_debitOrCredit', 'A_amount', 'A_valueDate', 'A_currencyCode', 'A_account', 'A_transactionReferences', 'A_transactionAttributes', 'A_orderingPartyInfo', 'A_receivingPartyInfo', 'generatedArtifactName', 'generatedId', 'generatedImportDate', 'B_debitOrCredit', 'B_amount', 'B_valueDate', 'B_currencyCode', 'B_account', 'B_transactionReferences', 'B_transactionAttributes', 'B_orderingPartyInfo', 'B_receivingPartyInfo', 'targetAllocation', 'generatorAllocation']


In [6]:
columns_A = ['matchId', 'matchDate', 'matchRule', 'matchedBy', 'wasPreviouslyMismatched', 'generatorArtifactName', 'generatorId', 'generatorImportDate', 'A_debitOrCredit', 'A_amount', 'A_valueDate', 'A_currencyCode', 'A_account', 'A_transactionReferences', 'A_transactionAttributes', 'A_orderingPartyInfo', 'A_receivingPartyInfo', 'targetAllocation', 'generatorAllocation']
columns_B = ['matchId', 'matchDate', 'matchRule', 'matchedBy', 'wasPreviouslyMismatched', 'generatorArtifactName', 'generatorId', 'generatorImportDate', 'B_debitOrCredit', 'B_amount', 'B_valueDate', 'B_currencyCode', 'B_account', 'B_transactionReferences', 'B_transactionAttributes', 'B_orderingPartyInfo', 'B_receivingPartyInfo', 'targetAllocation', 'generatorAllocation']

In [7]:
df_A = df[columns_A]
df_A.head()

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,A_debitOrCredit,A_amount,A_valueDate,A_currencyCode,A_account,A_transactionReferences,A_transactionAttributes,A_orderingPartyInfo,A_receivingPartyInfo,targetAllocation,generatorAllocation
0,1287,9/18/13,MANUAL,Jane Doe,0,A,1291.0,9/18/13,NONE,5481.21,9/11/13,USD,3446886485,NaN,600 cc AAB,NaN,NaN,MULT,USD_9/11/13_3446886485_600 cc AAB
1,1293,9/18/13,MANUAL,Jane Doe,0,A,1313.0,9/18/13,NONE,"34,233.97",9/17/13,USD,3445996130,NaN,211 AAB,NaN,NaN,MULT,USD_9/17/13_3445996130_211 AAB
2,1293,9/18/13,MANUAL,Jane Doe,0,A,1314.0,9/18/13,NONE,"32,643.27",9/17/13,USD,3445996130,NaN,209 AAB,NaN,NaN,MULT,USD_9/17/13_3445996130_209 AAB
3,1293,9/18/13,MANUAL,Jane Doe,0,A,1315.0,9/18/13,NONE,"36,897.54",9/17/13,USD,3445996130,NaN,205 AAB,NaN,NaN,MULT,USD_9/17/13_3445996130_205 AAB
4,1293,9/18/13,MANUAL,Jane Doe,0,A,1316.0,9/18/13,NONE,"31,444.64",9/17/13,USD,3445996130,NaN,202 AAB,NaN,NaN,MULT,USD_9/17/13_3445996130_202 AAB


In [8]:
df_B = df[columns_B]
df_B.head()

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,B_debitOrCredit,B_amount,B_valueDate,B_currencyCode,B_account,B_transactionReferences,B_transactionAttributes,B_orderingPartyInfo,B_receivingPartyInfo,targetAllocation,generatorAllocation
0,1287,9/18/13,MANUAL,Jane Doe,0,A,1291.0,9/18/13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/11/13_3446886485_600 cc AAB
1,1293,9/18/13,MANUAL,Jane Doe,0,A,1313.0,9/18/13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_211 AAB
2,1293,9/18/13,MANUAL,Jane Doe,0,A,1314.0,9/18/13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_209 AAB
3,1293,9/18/13,MANUAL,Jane Doe,0,A,1315.0,9/18/13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_205 AAB
4,1293,9/18/13,MANUAL,Jane Doe,0,A,1316.0,9/18/13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_202 AAB


In [10]:
df_A.shape


(223937, 19)

In [15]:
df_A = df_A[df_A['A_amount'].notna()]

df_A.head()

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,A_debitOrCredit,A_amount,A_valueDate,A_currencyCode,A_account,A_transactionReferences,A_transactionAttributes,A_orderingPartyInfo,A_receivingPartyInfo,targetAllocation,generatorAllocation
0,1287,9/18/13,MANUAL,Jane Doe,0,A,1291.0,9/18/13,NONE,5481.21,9/11/13,USD,3446886485,NaN,600 cc AAB,NaN,NaN,MULT,USD_9/11/13_3446886485_600 cc AAB
1,1293,9/18/13,MANUAL,Jane Doe,0,A,1313.0,9/18/13,NONE,"34,233.97",9/17/13,USD,3445996130,NaN,211 AAB,NaN,NaN,MULT,USD_9/17/13_3445996130_211 AAB
2,1293,9/18/13,MANUAL,Jane Doe,0,A,1314.0,9/18/13,NONE,"32,643.27",9/17/13,USD,3445996130,NaN,209 AAB,NaN,NaN,MULT,USD_9/17/13_3445996130_209 AAB
3,1293,9/18/13,MANUAL,Jane Doe,0,A,1315.0,9/18/13,NONE,"36,897.54",9/17/13,USD,3445996130,NaN,205 AAB,NaN,NaN,MULT,USD_9/17/13_3445996130_205 AAB
4,1293,9/18/13,MANUAL,Jane Doe,0,A,1316.0,9/18/13,NONE,"31,444.64",9/17/13,USD,3445996130,NaN,202 AAB,NaN,NaN,MULT,USD_9/17/13_3445996130_202 AAB


In [16]:
df_B = df_B[df_B['B_amount'].notna()]

df_B.head()

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,B_debitOrCredit,B_amount,B_valueDate,B_currencyCode,B_account,B_transactionReferences,B_transactionAttributes,B_orderingPartyInfo,B_receivingPartyInfo,targetAllocation,generatorAllocation
345,3,9/20/13,:000001,Jane Doe,0,A,3.0,9/18/13,NONE,"-2,310.84",9/12/13,USD,3.446051e+09,NaN,868.0,NaN,NaN,USD_9/13/13_3446050625_BOA Tra AAA,USD_9/13/13_3446050625_BOA Tra AAA
346,4,9/20/13,:000001,Jane Doe,0,A,4.0,9/18/13,NONE,"-4,518.81",9/2/13,USD,3.446205e+09,NaN,868.0,NaN,NaN,USD_9/6/13_3446205303_BOA Tra AAA,USD_9/6/13_3446205303_BOA Tra AAA
347,5,9/20/13,:000001,Jane Doe,0,A,5.0,9/18/13,NONE,"-1,502.62",9/5/13,USD,3.446854e+09,NaN,868.0,NaN,NaN,USD_9/6/13_3446853861_BOA Tra AAA,USD_9/6/13_3446853861_BOA Tra AAA
348,6,9/20/13,:000001,Jane Doe,0,A,6.0,9/18/13,NONE,5298.47,9/5/13,USD,9.286037e+08,NaN,868.0,NaN,NaN,USD_9/6/13_928603651_BOA Tra AAA,USD_9/6/13_928603651_BOA Tra AAA
349,8,9/20/13,:000001,Jane Doe,0,A,8.0,10/3/13,NONE,"-134,445.48",9/19/13,USD,3.446051e+09,NaN,868.0,NaN,NaN,USD_9/20/13_3446050638_BOA Tra AAA,USD_9/20/13_3446050638_BOA Tra AAA


In [17]:
print(df_A.shape)
print(df_B.shape)

(211744, 19)
(190717, 19)


#There are more A Transaction info then B transaction information